In [ ]:
!pip install ultralytics opencv-python

## Run it Before running any of the Below Code

In [1]:
import torch
print("CUDA Available:", torch.cuda.is_available())
print("Using device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")

CUDA Available: True
Using device: NVIDIA GeForce GTX 1650


#### CUDA donwload for 1650 gpu

In [ ]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

## Code with 4 different ROI' & Light Logic

In [13]:
# This is comment This is Basic First Code for Initializing Variables 
#second line comment
import os
import cv2
import numpy as np
from ultralytics import YOLO
import threading
import time

# --- USER: VERIFY YOUR PROJECT DIRECTORY ---
PROJECT_DIR = "/content/drive/MyDrive/Colab_Projects/AI-Based-Intelligent-Traffic-Monitoring-and-Management-System"
# The name of your input video file, which should be in the 'data' subfolder
INPUT_VIDEO_FILENAME = "input.mp4"
# ---

# --- USER: DEFINE YOUR LANE'S ROI HERE ---
# Define the coordinates of YOUR lane's polygon
ROI_POINTS = np.array([(150, 100), (600, 100), (600, 400), (150, 400)], np.int32)
# --- Construct ROIs for 4 webcams (can customize per webcam) ---
ROIS = {
    0: ROI_POINTS,
    1: ROI_POINTS,
    2: ROI_POINTS,
    3: ROI_POINTS
}

# Construct full paths for video files
input_video_path = os.path.join(PROJECT_DIR, 'data', INPUT_VIDEO_FILENAME)
output_video_path = os.path.join(PROJECT_DIR, 'output', 'output_vehicle_count.mp4')

# Ensure the output directory exists
os.makedirs(os.path.join(PROJECT_DIR, 'output'), exist_ok=True)

# Change the current working directory to the project folder
os.chdir(PROJECT_DIR)

# Load the pre-trained YOLOv8 model
print("Loading YOLOv8 model...")
model = YOLO("yolov8n.pt").to("cuda")

# COCO class IDs for vehicles (car, motorcycle, bus, truck)
VEHICLE_CLASS_IDS = [2, 3, 5, 7]

# --- Shared dictionary for real-time counts ---
lane_counts = {0: 0, 1: 0, 2: 0, 3: 0}  # each webcam updates its count
stop_program = False  # global flag to stop all threads

# --- Traffic light parameters ---
min_green = 10
max_green = 30
max_count = 20

# --- Webcam processing function ---
def process_webcam(cam_index, window_name):
    global stop_program
    cap = cv2.VideoCapture(cam_index, cv2.CAP_DSHOW)

    if not cap.isOpened():
        print(f"❌ Error: Could not open webcam {cam_index}")
        return

    roi_polygon = ROIS[cam_index]

    while not stop_program:
        success, frame = cap.read()
        if not success:
            break

        vehicle_count = 0
        cv2.polylines(frame, [roi_polygon], isClosed=True, color=(255, 0, 0), thickness=2)

        results = model.predict(frame, device=0, verbose=False)
        for box in results[0].boxes:
            cls_id = int(box.cls)
            conf = float(box.conf[0])
            if cls_id in VEHICLE_CLASS_IDS and conf > 0.5:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2
                if cv2.pointPolygonTest(roi_polygon, (center_x, center_y), False) >= 0:
                    vehicle_count += 1
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # update shared dictionary
        lane_counts[cam_index] = vehicle_count

        cv2.putText(frame, f"Vehicles: {vehicle_count}", (20, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        cv2.imshow(window_name, frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            stop_program = True  # set flag to stop all threads
            break

    cap.release()
    cv2.destroyWindow(window_name)

# --- Dynamic Traffic Light Controller ---
def traffic_controller():
    global stop_program
    while not stop_program:
        for lane_id in range(4):
            if stop_program:
                break
            count = lane_counts[lane_id]
            green_time = int(min_green + (count / max_count) * (max_green - min_green))
            green_time = max(min_green, min(green_time, max_green))

            print(f"\n🚦 Lane {lane_id} → GREEN for {green_time}s (Vehicles: {count})")
            print("Other lanes → RED")

            for sec in range(green_time, 0, -1):
                if stop_program:
                    break
                print(f"   Lane {lane_id} green → {sec} sec left", end="\r")
                time.sleep(1)

# --- Run 4 webcams + traffic controller ---
threads = []
for i in range(4):
    t = threading.Thread(target=process_webcam, args=(i, f"Webcam {i}"))
    threads.append(t)
    t.start()

controller_thread = threading.Thread(target=traffic_controller, daemon=True)
controller_thread.start()

for t in threads:
    t.join()

cv2.destroyAllWindows()


Loading YOLOv8 model...

🚦 Lane 0 → GREEN for 10s (Vehicles: 0)
Other lanes → RED
Ultralytics 8.3.200  Python-3.11.9 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)
Ultralytics 8.3.200  Python-3.11.9 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)
YOLOv8n summary (fused): 72 layers, 3,151,904 parameters, 0 gradients, 8.7 GFLOPs
YOLOv8n summary (fused): 72 layers, 3,151,904 parameters, 0 gradients, 8.7 GFLOPs
